# Etape 2.2 : Fusion et enrichissement

**Livrables** :
- Ce notebook `05_fusion_enrichissement.ipynb`
- Dataset final `output/05_consommations_enrichies.csv` et `output/05_consommations_enrichies.parquet`
- Dictionnaire de donnees (description de toutes les colonnes)

---
---

## Import

In [1]:
import sys
import os
from pathlib import Path
import psutil
import time
from datetime import datetime
import warnings 
import pandas as pd

---

## (optionnel) Enregistrement de la date de la dernière execution de ce notebook

In [2]:
print(f"- Date de la dernière execution de ce notebook : {datetime.now().strftime('%d/%m/%Y %H:%M:%S')} (FR)")

- Date de la dernière execution de ce notebook : 20/02/2026 20:33:00 (FR)


---

## (Optionnel) Mesure du temps de traitement global pour ce script - enregistrement de l'heure de début + estimation instantanée des ressources machine libres

In [3]:
## Heure de début
start_time_05 = time.time()

## Machine: current available RAM (in GB)
ram_available_05 = psutil.virtual_memory().available / (1024**3)

## Machine: current available CPU
logical = psutil.cpu_count()
physical = psutil.cpu_count(logical=False) or logical

cpu_used = psutil.cpu_percent(interval=2)
cpu_available_pct_05 = 100 - cpu_used

available_logical_05 = logical * cpu_available_pct_05 / 100
available_physical_05 = physical * cpu_available_pct_05 / 100

## Show available resources
print(f"- Current machine RAM available : {ram_available_05:.2f} GB")
print(f"- Current machine CPU available : {cpu_available_pct_05:.2f}%")
print(f"    Approx logical cores free  : {available_logical_05:.1f}")
print(f"    Approx physical cores free : {available_physical_05:.1f}")

- Current machine RAM available : 10.68 GB
- Current machine CPU available : 81.70%
    Approx logical cores free  : 13.1
    Approx physical cores free : 6.5


---

## Utils

In [4]:
# --- Fnc. pour savoir la qualité (completude, ...) des dataframes
def quality_report(df):
    print(f"    - Lignes: {len(df):,}")
    print(f"    - Colonnes: {len(df.columns)}")
    
    report = []
    for col in df.columns:
        total = len(df)
        missing = df[col].isna().sum() + (df[col] == '').sum() if df[col].dtype == 'object' else df[col].isna().sum()
        completude = (1 - missing / total) * 100
        unique = df[col].nunique()
        dtype = df[col].dtype
        
        report.append({
            'Colonne': col,
            'Type': str(dtype),
            'Manquants': missing,
            'Completude %': round(completude, 2),
            'Uniques': unique
        })
    
    return pd.DataFrame(report)

# --- Python cfg.: 
# cacher (ignorer) les warnings
warnings.filterwarnings('ignore') 

# --- Pandas cfg.:
# show ALL rows
pd.set_option('display.max_rows', None)
# show ALL columns
pd.set_option('display.max_columns', None)
# prevent column text from being truncated
pd.set_option('display.max_colwidth', None)

---

## Chemins des données

In [5]:
# ==============================================================================================================
#                                                   OUTPUTS
# ==============================================================================================================
OUT_DIR = (Path.cwd() / ".." / "output").resolve()
OUT_CONSO_ENRICHIES_CSV =  os.path.join(OUT_DIR, "05_consommations_enrichies.csv")
OUT_CONSO_ENRICHIES_PARQUET = os.path.join(OUT_DIR, "05_consommations_enrichies.parquet")

# ==============================================================================================================
#                                                   INPUTS
# ==============================================================================================================
IN_DIR = (Path.cwd() / ".." / "data").resolve()
IN_CONSO_CLEAN_PARQUET = os.path.join(OUT_DIR, "02_consommations_clean")
IN_METEO_CLEAN_CSV = os.path.join(OUT_DIR, "04_meteo_clean.csv")
IN_BAT_CSV = os.path.join(IN_DIR, "batiments.csv")
IN_TARIF_CSV = os.path.join(IN_DIR, "tarifs_energie.csv")

# ==============================================================================================================
#                                                    OTHERS
# ==============================================================================================================
TMP_DIR = (Path.cwd() / ".." / "my_tmp").resolve()
TMP_FILE_CSV = TMP_DIR / "tmp_05_resources.txt" # Enregistrer les metrics pour ce script

---

## Charger les consommations nettoyees (depuis Parquet)

In [6]:
df_conso = pd.read_parquet(IN_CONSO_CLEAN_PARQUET)

## Affichage de quelques infos
print("df_conso :")
print(f"    - Shape: {df_conso.shape}")
print(f"    - Colonnes: {df_conso.columns.tolist()}")
print()
# Info sur les types
print("    - Infos sur les types : ")
df_conso.info()
print()
## Appercu des donnees
print("    - Appercu des donnees : ")
df_conso.head()

df_conso :
    - Shape: (991, 6)
    - Colonnes: ['batiment_id', 'heure', 'consommation_moyenne', 'unite', 'date', 'type_energie']

    - Infos sur les types : 
<class 'pandas.DataFrame'>
RangeIndex: 991 entries, 0 to 990
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   batiment_id           991 non-null    str     
 1   heure                 991 non-null    int32   
 2   consommation_moyenne  991 non-null    float64 
 3   unite                 991 non-null    str     
 4   date                  991 non-null    category
 5   type_energie          991 non-null    category
dtypes: category(2), float64(1), int32(1), str(2)
memory usage: 49.1 KB

    - Appercu des donnees : 


,batiment_id,heure,consommation_moyenne,unite,date,type_energie
0,BAT0043,22,280.88,m3,2023-01-01,eau
1,BAT0100,17,4.52,m3,2023-01-01,eau
2,BAT0006,3,5.50,kWh,2023-01-01,gaz
3,BAT0126,10,126.88,kWh,2023-01-01,gaz
4,BAT0035,20,3.97,m3,2023-01-02,eau


---

## Charger les données météo nettoyees

In [7]:
df_meteo = pd.read_csv(IN_METEO_CLEAN_CSV)

## Affichage de quelques infos
print("df_meteo :")
print(f"    - Shape: {df_meteo.shape}")
print(f"    - Colonnes: {df_meteo.columns.tolist()}")
print()
# Info sur les types
print("    - Infos sur les types : ")
df_meteo.info()
print()
## Appercu des donnees
print("    - Appercu des donnees : ")
df_meteo.head()


df_meteo :
    - Shape: (236232, 11)
    - Colonnes: ['commune', 'ts_h', 'temperature_c', 'humidite_pct', 'rayonnement_solaire_wm2', 'vitesse_vent_kmh', 'precipitation_mm', 'jour', 'mois', 'saison', 'jour_de_semaine']

    - Infos sur les types : 
<class 'pandas.DataFrame'>
RangeIndex: 236232 entries, 0 to 236231
Data columns (total 11 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   commune                  236232 non-null  str    
 1   ts_h                     236232 non-null  str    
 2   temperature_c            236232 non-null  float64
 3   humidite_pct             236232 non-null  float64
 4   rayonnement_solaire_wm2  236232 non-null  float64
 5   vitesse_vent_kmh         236232 non-null  float64
 6   precipitation_mm         236232 non-null  float64
 7   jour                     236232 non-null  str    
 8   mois                     236232 non-null  int64  
 9   saison                   236232 non-null  

,commune,ts_h,temperature_c,humidite_pct,rayonnement_solaire_wm2,vitesse_vent_kmh,precipitation_mm,jour,mois,saison,jour_de_semaine
0,Bordeaux,2023-01-01 00:00:00,8.5,43.9,43.9,0.2,0.0,2023-01-01,1,Hiver,6
1,Bordeaux,2023-01-01 01:00:00,2.7,39.7,39.7,21.5,5.3,2023-01-01,1,Hiver,6
2,Bordeaux,2023-01-01 02:00:00,6.2,78.9,78.9,13.1,0.0,2023-01-01,1,Hiver,6
3,Bordeaux,2023-01-01 03:00:00,10.3,64.2,64.2,0.6,14.7,2023-01-01,1,Hiver,6
4,Bordeaux,2023-01-01 04:00:00,0.9,36.4,36.4,35.6,0.0,2023-01-01,1,Hiver,6


---

## Chargement du referenciel batiments

In [8]:
df_bat = pd.read_csv(IN_BAT_CSV)

## Affichage de quelques infos
print("df_bat :")
print(f"    - Shape: {df_bat.shape}")
print(f"    - Colonnes: {df_bat.columns.tolist()}")
print()
# Info sur les types
print("    - Infos sur les types : ")
df_bat.info()
print()
## Appercu des donnees
print("    - Appercu des donnees : ")
df_bat.head()

df_bat :
    - Shape: (146, 8)
    - Colonnes: ['batiment_id', 'nom', 'type', 'commune', 'surface_m2', 'annee_construction', 'classe_energetique', 'nb_occupants_moyen']

    - Infos sur les types : 
<class 'pandas.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   batiment_id         146 non-null    str  
 1   nom                 146 non-null    str  
 2   type                146 non-null    str  
 3   commune             146 non-null    str  
 4   surface_m2          146 non-null    int64
 5   annee_construction  146 non-null    int64
 6   classe_energetique  146 non-null    str  
 7   nb_occupants_moyen  146 non-null    int64
dtypes: int64(3), str(5)
memory usage: 15.1 KB

    - Appercu des donnees : 


,batiment_id,nom,type,commune,surface_m2,annee_construction,classe_energetique,nb_occupants_moyen
0,BAT0001,Ecole Paris 1,ecole,Paris,1926,1978,E,225
1,BAT0002,Ecole Paris 2,ecole,Paris,1156,2004,C,402
2,BAT0003,Ecole Paris 3,ecole,Paris,1695,2014,D,219
3,BAT0004,Mediatheque Paris 4,mediatheque,Paris,907,2019,C,121
4,BAT0005,Piscine Paris 5,piscine,Paris,3913,1950,G,242


---

## Chargement des tarifs

In [9]:
df_tarifs = pd.read_csv(IN_TARIF_CSV)

## Affichage de quelques infos
print("df_tarifs :")
print(f"    - Shape: {df_tarifs.shape}")
print(f"    - Colonnes: {df_tarifs.columns.tolist()}")
print()
# Info sur les types
print("    - Infos sur les types : ")
df_tarifs.info()
print()
## Appercu des donnees
print("    - Appercu des donnees : ")
# df_tarifs.head()
print(df_tarifs)

df_tarifs :
    - Shape: (10, 4)
    - Colonnes: ['date_debut', 'date_fin', 'type_energie', 'tarif_unitaire']

    - Infos sur les types : 
<class 'pandas.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date_debut      10 non-null     str    
 1   date_fin        10 non-null     str    
 2   type_energie    10 non-null     str    
 3   tarif_unitaire  10 non-null     float64
dtypes: float64(1), str(3)
memory usage: 714.0 bytes

    - Appercu des donnees : 
   date_debut    date_fin type_energie  tarif_unitaire
0  2023-01-01  2023-06-30  electricite            0.18
1  2023-07-01  2023-12-31  electricite            0.20
2  2023-01-01  2023-06-30          gaz            0.09
3  2023-07-01  2023-12-31          gaz            0.10
4  2023-01-01  2023-12-31          eau            3.50
5  2024-01-01  2024-06-30  electricite            0.21
6  2024-07-01  2024-12-31  elect

---

## Fusionner des données de consommation (clean) avec les donnees meteo (sur commune et timestamp arrondi a l'heure)

df_conso & df_meteo n'ont pas pour le momment de colonnes communes. Donc les étapes de la fusion (après le traitemet des timestamps) seront :
1. Fusion de df_conso & df_bat pour obtenir 'commune' (result: *df_conso_bat*)
2. Fusion avec df_meteo (result: *df*)

In [10]:
# --- Traitement des timestamps (de df_conso & df_meteo)
df_conso["date"] = pd.to_datetime(df_conso["date"])
df_conso["heure"] = pd.to_numeric(df_conso["heure"], errors="coerce").astype("Int64")
    # Construire ts_h côté conso (date + heure)
df_conso["ts_h"] = pd.to_datetime(df_conso['date'].astype(str) + ' ' + df_conso['heure'].astype(str) + ':00:00')

# df_meteo (just to be sure)
df_meteo["ts_h"] = pd.to_datetime(df_meteo["ts_h"])

# --- Check de la qualité des données des df_conso & df_meteo
# df_conso :
print("Check de la qualité des données des df_conso")
df_report_conso = quality_report(df_conso)
print("    df_report_conso : ")
print(df_report_conso)

cols_non_zero = df_report_conso.loc[
    df_report_conso["Manquants"] != 0, "Colonne"
].tolist()

if len(cols_non_zero) != 0:
    print("    [ko]: les colonnes avec des valeurs manquantes : ", cols_non_zero)
    sys.exit(0)
else:
    print("    [ok]: pas de colonnes avec des valeurs manquantes dans df_conso")

# df_meteo :
print()
print("Check de la qualité des données des df_meteo")
df_report_meteo = quality_report(df_meteo)
print("    df_report_meteo : ")
print(df_report_meteo)

cols_non_zero = df_report_meteo.loc[
    df_report_meteo["Manquants"] != 0, "Colonne"
].tolist()

if len(cols_non_zero) != 0:
    print("    [ko]: les colonnes avec des valeurs manquantes : ", cols_non_zero)
    sys.exit(0)
else:
    print("    [ok]: pas de colonnes avec des valeurs manquantes dans df_meteo")



Check de la qualité des données des df_conso
    - Lignes: 991
    - Colonnes: 7
    df_report_conso : 
                Colonne            Type  Manquants  Completude %  Uniques
0           batiment_id             str          0         100.0      146
1                 heure           Int64          0         100.0       24
2  consommation_moyenne         float64          0         100.0      947
3                 unite             str          0         100.0        2
4                  date        category          0         100.0      538
5          type_energie        category          0         100.0        3
6                  ts_h  datetime64[us]          0         100.0      958
    [ok]: pas de colonnes avec des valeurs manquantes dans df_conso

Check de la qualité des données des df_meteo
    - Lignes: 236,232
    - Colonnes: 11
    df_report_meteo : 
                    Colonne            Type  Manquants  Completude %  Uniques
0                   commune             str     

In [11]:
# --- Fusion de df_conso & df_bat pour obtenir 'commune'
df_conso_bat = df_conso.merge(df_bat, on="batiment_id", how="left")

# check
print()
if (df_conso_bat["commune"].isna().sum() != 0):
    print("[ko]: Batiments non matchés :", df_conso_bat["commune"].isna().sum()) 
    sys.exit(1)
else:
    print("[ok]: Batiments non matchés :", df_conso_bat["commune"].isna().sum()) 

# Apperçu de df_conso_bat
print()
print("Apperçu de df_conso_bat :")
# df_conso_bat.head()
df_conso_bat.info()


[ok]: Batiments non matchés : 0

Apperçu de df_conso_bat :
<class 'pandas.DataFrame'>
RangeIndex: 991 entries, 0 to 990
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   batiment_id           991 non-null    str           
 1   heure                 991 non-null    Int64         
 2   consommation_moyenne  991 non-null    float64       
 3   unite                 991 non-null    str           
 4   date                  991 non-null    category      
 5   type_energie          991 non-null    category      
 6   ts_h                  991 non-null    datetime64[us]
 7   nom                   991 non-null    str           
 8   type                  991 non-null    str           
 9   commune               991 non-null    str           
 10  surface_m2            991 non-null    int64         
 11  annee_construction    991 non-null    int64         
 12  classe_energetique    991 non

In [12]:
# --- Fusion (de df_conso_bat) avec df_meteo
df_conso_meteo = df_conso_bat.merge(df_meteo, on=["commune", "ts_h"], how="left")

print("df_conso_meteo : ")
df_conso_meteo.info()

df_conso_meteo : 
<class 'pandas.DataFrame'>
RangeIndex: 991 entries, 0 to 990
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   batiment_id              991 non-null    str           
 1   heure                    991 non-null    Int64         
 2   consommation_moyenne     991 non-null    float64       
 3   unite                    991 non-null    str           
 4   date                     991 non-null    category      
 5   type_energie             991 non-null    category      
 6   ts_h                     991 non-null    datetime64[us]
 7   nom                      991 non-null    str           
 8   type                     991 non-null    str           
 9   commune                  991 non-null    str           
 10  surface_m2               991 non-null    int64         
 11  annee_construction       991 non-null    int64         
 12  classe_energetique       991 

In [13]:
print("Aperçu de df_conso_meteo : ")
df_conso_meteo.head()

Aperçu de df_conso_meteo : 


,batiment_id,heure,consommation_moyenne,unite,date,type_energie,ts_h,nom,type,commune,surface_m2,annee_construction,classe_energetique,nb_occupants_moyen,temperature_c,humidite_pct,rayonnement_solaire_wm2,vitesse_vent_kmh,precipitation_mm,jour,mois,saison,jour_de_semaine
0,BAT0043,22,280.88,m3,2023-01-01,eau,2023-01-01 22:00:00,Piscine Bordeaux 43,piscine,Bordeaux,2279,1962,G,148,8.4,68.7,68.7,38.9,0.0,2023-01-01,1.0,Hiver,6.0
1,BAT0100,17,4.52,m3,2023-01-01,eau,2023-01-01 17:00:00,Mediatheque Rennes 100,mediatheque,Rennes,898,1999,E,121,-0.2,87.1,87.1,8.2,7.6,2023-01-01,1.0,Hiver,6.0
2,BAT0006,3,5.50,kWh,2023-01-01,gaz,2023-01-01 03:00:00,Mairie Paris 6,mairie,Paris,1165,1985,E,63,2.3,59.7,59.7,25.4,0.0,2023-01-01,1.0,Hiver,6.0
3,BAT0126,10,126.88,kWh,2023-01-01,gaz,2023-01-01 10:00:00,Ecole Le Havre 126,ecole,Le Havre,1692,1982,G,338,-0.3,52.4,52.4,26.8,0.0,2023-01-01,1.0,Hiver,6.0
4,BAT0035,20,3.97,m3,2023-01-02,eau,2023-01-02 20:00:00,Mairie Toulouse 35,mairie,Toulouse,842,2004,F,87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Infos suplementaire (Audit) : 
print("Météo manquante (%) :", (df_conso_meteo["temperature_c"].isna().mean() * 100).round(2))

Météo manquante (%) : 10.8


---

## Fusionner avec les tarifs pour calculer le cout financier

In [15]:
# --- Assurer les types
df = df_conso_meteo.copy()  
df["date"] = pd.to_datetime(df["date"])

df_tarifs2 = df_tarifs.copy()
df_tarifs2["date_debut"] = pd.to_datetime(df_tarifs2["date_debut"])
df_tarifs2["date_fin"]   = pd.to_datetime(df_tarifs2["date_fin"])

# --- Merge sur type_energie (ça duplique par périodes)
df_out = df.merge(df_tarifs2, on="type_energie", how="left")

# --- Forcer les types datetime (même si elles sont "category")
for c in ["date", "date_debut", "date_fin"]:
    df_out[c] = pd.to_datetime(df_out[c].astype(str), errors="coerce")

# --- Garder la bonne période tarifaire
# Filtrer la bonne période
df_out = df_out[
    (df_out["date"] >= df_out["date_debut"]) &
    (df_out["date"] <= df_out["date_fin"])
]

# --- Calcul du coût
df_out["cout_financier"] = (df_out["consommation_moyenne"] * df_out["tarif_unitaire"]).round(3)

df_out.head()

,batiment_id,heure,consommation_moyenne,unite,date,type_energie,ts_h,nom,type,commune,surface_m2,annee_construction,classe_energetique,nb_occupants_moyen,temperature_c,humidite_pct,rayonnement_solaire_wm2,vitesse_vent_kmh,precipitation_mm,jour,mois,saison,jour_de_semaine,date_debut,date_fin,tarif_unitaire,cout_financier
0,BAT0043,22,280.88,m3,2023-01-01,eau,2023-01-01 22:00:00,Piscine Bordeaux 43,piscine,Bordeaux,2279,1962,G,148,8.4,68.7,68.7,38.9,0.0,2023-01-01,1.0,Hiver,6.0,2023-01-01,2023-12-31,3.50,983.080
2,BAT0100,17,4.52,m3,2023-01-01,eau,2023-01-01 17:00:00,Mediatheque Rennes 100,mediatheque,Rennes,898,1999,E,121,-0.2,87.1,87.1,8.2,7.6,2023-01-01,1.0,Hiver,6.0,2023-01-01,2023-12-31,3.50,15.820
4,BAT0006,3,5.50,kWh,2023-01-01,gaz,2023-01-01 03:00:00,Mairie Paris 6,mairie,Paris,1165,1985,E,63,2.3,59.7,59.7,25.4,0.0,2023-01-01,1.0,Hiver,6.0,2023-01-01,2023-06-30,0.09,0.495
8,BAT0126,10,126.88,kWh,2023-01-01,gaz,2023-01-01 10:00:00,Ecole Le Havre 126,ecole,Le Havre,1692,1982,G,338,-0.3,52.4,52.4,26.8,0.0,2023-01-01,1.0,Hiver,6.0,2023-01-01,2023-06-30,0.09,11.419
12,BAT0035,20,3.97,m3,2023-01-02,eau,2023-01-02 20:00:00,Mairie Toulouse 35,mairie,Toulouse,842,2004,F,87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-01,2023-12-31,3.50,13.895


---

---

## Creer des features derivees :
  - Consommation par occupant
  - Consommation par m2
  - Cout journalier, mensuel, annuel
  - Indice de performance energetique (IPE)
  - Ecart a la moyenne de la categorie

In [16]:
df_enrichie = df_out.copy()
df_enrichie["conso_par_occupant"] = (df_enrichie["consommation_moyenne"] / df_enrichie["nb_occupants_moyen"]).round(3)
df_enrichie["conso_par_m2"] = (df_enrichie["consommation_moyenne"] / df_enrichie["surface_m2"]).round(3)
df_enrichie["IPE"] = (df_enrichie["conso_par_m2"]).round(3)

df_enrichie["ecart_moyenne_categorie"] = (
    (df_enrichie["consommation_moyenne"] -
    df_enrichie.groupby("type")["consommation_moyenne"].transform("mean")).round(3)
)

df_enrichie.head()

,batiment_id,heure,consommation_moyenne,unite,date,type_energie,ts_h,nom,type,commune,surface_m2,annee_construction,classe_energetique,nb_occupants_moyen,temperature_c,humidite_pct,rayonnement_solaire_wm2,vitesse_vent_kmh,precipitation_mm,jour,mois,saison,jour_de_semaine,date_debut,date_fin,tarif_unitaire,cout_financier,conso_par_occupant,conso_par_m2,IPE,ecart_moyenne_categorie
0,BAT0043,22,280.88,m3,2023-01-01,eau,2023-01-01 22:00:00,Piscine Bordeaux 43,piscine,Bordeaux,2279,1962,G,148,8.4,68.7,68.7,38.9,0.0,2023-01-01,1.0,Hiver,6.0,2023-01-01,2023-12-31,3.50,983.080,1.898,0.123,0.123,-401.254
2,BAT0100,17,4.52,m3,2023-01-01,eau,2023-01-01 17:00:00,Mediatheque Rennes 100,mediatheque,Rennes,898,1999,E,121,-0.2,87.1,87.1,8.2,7.6,2023-01-01,1.0,Hiver,6.0,2023-01-01,2023-12-31,3.50,15.820,0.037,0.005,0.005,-92.701
4,BAT0006,3,5.50,kWh,2023-01-01,gaz,2023-01-01 03:00:00,Mairie Paris 6,mairie,Paris,1165,1985,E,63,2.3,59.7,59.7,25.4,0.0,2023-01-01,1.0,Hiver,6.0,2023-01-01,2023-06-30,0.09,0.495,0.087,0.005,0.005,-43.286
8,BAT0126,10,126.88,kWh,2023-01-01,gaz,2023-01-01 10:00:00,Ecole Le Havre 126,ecole,Le Havre,1692,1982,G,338,-0.3,52.4,52.4,26.8,0.0,2023-01-01,1.0,Hiver,6.0,2023-01-01,2023-06-30,0.09,11.419,0.375,0.075,0.075,7.068
12,BAT0035,20,3.97,m3,2023-01-02,eau,2023-01-02 20:00:00,Mairie Toulouse 35,mairie,Toulouse,842,2004,F,87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-01,2023-12-31,3.50,13.895,0.046,0.005,0.005,-44.816


---

## Creation de consommations_enrichies.csv & consommations_enrichies.parquet

In [17]:
conso_enrichie_csv = os.path.join(OUT_DIR,"05_consommations_enrichies.csv")
conso_enrichie_parquet = os.path.join(OUT_DIR,"05_consommations_enrichies.parquet")

## Écriture en CSV
df_enrichie.to_csv(conso_enrichie_csv, index=False)

## Écriture en Parquet
df_enrichie.to_parquet(
    conso_enrichie_parquet,
    engine="pyarrow",
    index=False
)

---

## Libérer la mémoire (Optionnel) 

In [18]:
del df
del df_conso_meteo
del df_bat
del df_conso
del df_enrichie
del df_out

---

## (Optionnel) enregistrement dans un fichier temporaire du temps d'execution + ressources pour utilisation ultérieure (dans le script run_pipeline_hybride.py ou autres)

In [19]:
temps_execution_05 = time.time() - start_time_05
temps_resources = f"""
    Date : {datetime.now().strftime("%d/%m/%Y %H:%M:%S")} (FR)

    temps_exec_sec={temps_execution_05:.2f}
    ram_gb={ram_available_05:.2f}
    cpu_pct={cpu_available_pct_05:.2f}
    logi_cores={available_logical_05:.1f}
    physi_cores={available_physical_05:.1f}
"""

# Ecrire des données du temps d'execution + ressources dans le fichier TMP_FILE_CSV
TMP_FILE_CSV.write_text(temps_resources, encoding="utf-8")

137